# Project 2 Classification of Hsp70 proteins into taxonomic groups based on their amino acid sequences
 This notebook is only to compute the performance of each model provide in the report. Please refere the other notebook `Notebook_MTP_project2` for a description of the code and complete access to the code of this project.

## Import librairies

In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np

%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, make_scorer

from create_batch import *
from datatreatment import *
from neuralnet import *
from visualisations import *
from mutual_info import *
from TSNE import *
from train_rdf import *
from rdf_hyperpara_opti import *
from nn_hyperpara_opti import *

## Importing and treating the data set

In [5]:
path_to_dataset ="C:\\Users\\pierr\\OneDrive\\Bureau\\Projet_ML\\dataset_hsp70_tax.csv"
hsp70 = importing_data(path_to_dataset)

In [6]:
level3, level3_categ= get_data(hsp70, 3, 4,"Eukaryota")
level3=encode01(level3)
level3=category_to_int(level3,level3_categ)

number of Opisthokonta : 
{25989}
---------
number of Viridiplantae : 
{6861}
---------
number of Sar : 
{2241}
---------
number of Others : 
{1286}
---------
shape of dataframe : (36377, 12834)


## Mutual information

In [ ]:
mutual_data = np.loadtxt('mutual_data.txt',dtype=float)

## Random Forest

In [9]:
#best score
best_para = ({'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': False, 'class_weight': 'balanced', 'min_samples_leaf': 5}, 0, 0)
train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9,0.1,0)

F1 =[]
for i in range(30):
    model_rdf = train_random_forest(train, train_label,test, test_label,best_para[0])
    test_pred = model_rdf.predict(test)
    F1.append(f1_score(test_label, test_pred, average='weighted'))

print(f"F1 score of the random forest: {np.mean(F1)} with std: {np.std(F1)})

Parameters are: {'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': False, 'class_weight': 'balanced', 'min_samples_leaf': 5}
Training done in 146.54566621780396
Parameters are: {'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': False, 'class_weight': 'balanced', 'min_samples_leaf': 5}
Training done in 147.403826713562
[0.9919688479825668, 0.9908731604541506]
0.000547843764208078
0.9914210042183587


In [ ]:
#with bootstrap
best_para = ({'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': True, 'class_weight': 'balanced', 'min_samples_leaf': 5}, 0, 0)
train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9,0.1,0)

F1 =[]
for i in range(30):
    model_rdf = train_random_forest(train, train_label,test, test_label,best_para[0])
    test_pred = model_rdf.predict(test)
    F1.append(f1_score(test_label, test_pred, average='weighted'))

print(f"F1 score of the random forest: {np.mean(F1)} with std: {np.std(F1)})

In [ ]:
#without 'balanced' option
best_para = ({'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': False, 'min_samples_leaf': 5}, 0, 0)
train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9,0.1,0)

F1 =[]
for i in range(30):
    model_rdf = train_random_forest(train, train_label,test, test_label,best_para[0])
    test_pred = model_rdf.predict(test)
    F1.append(f1_score(test_label, test_pred, average='weighted'))

print(f"F1 score of the random forest: {np.mean(F1)} with std: {np.std(F1)})

## Neural Net

In [85]:
# best score
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0.8, 'l2_regu': 1e-05, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
input_dim = train.shape[1]
output_dim = 4

F1 =[]
for i in range(30):
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    test_outputs = model_neural(test)
    test_pred = test_outputs.argmax(dim=1)
    F1.append(f1_score(test_label, test_pred, average='weighted'))
print(F1)
print(np.std(F1))
print(np.mean(F1))

In [111]:
#without dropout
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0, 'l2_regu': 1e-05, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
input_dim = train.shape[1]
output_dim = 4

F1 =[]
for i in range(30):
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    test_outputs = model_neural(test)
    test_pred = test_outputs.argmax(dim=1)
    F1.append(f1_score(test_label, test_pred, average='weighted'))
print(F1)
print(np.std(F1))
print(np.mean(F1))

Epoch [1/10], Training Loss: 0.0628
Epoch [1/10], Validation Loss: 0.0733
Epoch [1/10], Validation Accuracy: 97.5529%
Epoch [2/10], Training Loss: 0.1155
Epoch [2/10], Validation Loss: 0.0493
Epoch [2/10], Validation Accuracy: 98.8177%
Epoch [3/10], Training Loss: 0.0598
Epoch [3/10], Validation Loss: 0.0405
Epoch [3/10], Validation Accuracy: 98.8727%
Epoch [4/10], Training Loss: 0.0553
Epoch [4/10], Validation Loss: 0.0350
Epoch [4/10], Validation Accuracy: 99.1202%
Epoch [5/10], Training Loss: 0.0205
Epoch [5/10], Validation Loss: 0.0374
Epoch [5/10], Validation Accuracy: 99.1202%
Epoch [6/10], Training Loss: 0.0135
Epoch [6/10], Validation Loss: 0.0381
Epoch [6/10], Validation Accuracy: 98.9827%
Epoch [7/10], Training Loss: 0.0589
Epoch [7/10], Validation Loss: 0.0364
Epoch [7/10], Validation Accuracy: 99.1751%
Epoch [8/10], Training Loss: 0.0769
Epoch [8/10], Validation Loss: 0.0395
Epoch [8/10], Validation Accuracy: 99.0652%
Epoch [9/10], Training Loss: 0.0244
Epoch [9/10], Valida

In [ ]:
#without l2-rugularization
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0.8, 'l2_regu': 0, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
input_dim = train.shape[1]
output_dim = 4

F1 =[]
for i in range(30):
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    test_outputs = model_neural(test)
    test_pred = test_outputs.argmax(dim=1)
    F1.append(f1_score(test_label, test_pred, average='weighted'))
print(F1)
print(np.std(F1))
print(np.mean(F1))

In [ ]:
#without weight_decay
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0.8, 'l2_regu': 1e-05, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
input_dim = train.shape[1]
output_dim = 4

F1 =[]
for i in range(30):
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    test_outputs = model_neural(test)
    test_pred = test_outputs.argmax(dim=1)
    F1.append(f1_score(test_label, test_pred, average='weighted'))
print(F1)
print(np.std(F1))
print(np.mean(F1))

In [ ]:
#without batches
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0.8, 'l2_regu': 1e-05, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
batch_size = train.shape[0]
input_dim = train.shape[1]
output_dim = 4

F1 =[]
for i in range(30):
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    test_outputs = model_neural(test)
    test_pred = test_outputs.argmax(dim=1)
    F1.append(f1_score(test_label, test_pred, average='weighted'))
print(F1)
print(np.std(F1))
print(np.mean(F1))

### Features importances

In [ ]:
best_params_nn = {'layer_dim': 256, 'number_hidden_layer': 4, 'dropout_prob': 0.8, 'l2_regu': 1e-05, 'weight_decay': 0.0001, 'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 10}
layer_dim = best_params_nn['layer_dim']
number_hidden_layer = best_params_nn['number_hidden_layer']
dropout_prob = best_params_nn['dropout_prob']
l2_regu = best_params_nn['l2_regu']
weight_decay = best_params_nn['weight_decay']
learning_rate = best_params_nn['learning_rate']
batch_size = best_params_nn['batch_size']
num_epochs = best_params_nn['num_epochs']

best_para = ({'n_estimators': 100, 'max_depth': 400, 'max_features': 400, 'bootstrap': False, 'class_weight': 'balanced', 'min_samples_leaf': 5}, 0, 0)

dfs = []
for i in range(30):
    
    train, train_label, test, test_label, val, val_label=split_dataset(level3, 0.9, 0.1, 0)
    
    model_rdf = train_random_forest(train, train_label,test, test_label,best_para[0])
    model_neural = ModelClassification(input_dim, output_dim, layer_dim, number_hidden_layer, dropout_prob, l2_regu)
    optimizer = torch.optim.Adam(model_neural.parameters(), lr = learning_rate, weight_decay=weight_decay)#lr : learning rate
    train_model(model_neural, num_epochs, train, train_label, test, test_label, optimizer, batch_size)
    
    dfs.append(feature_importances(model_rdf, model_neural, mutual_data, level3, smoothness = 40, plot = 0))
    
    
    

arrays = [df.to_numpy() for df in dfs]

stacked_array = np.stack(arrays, axis=0)
mean_values = np.mean(stacked_array, axis=0)
std_values = np.std(stacked_array, axis=0)


np.savetxt('mean.txt', mean_values)
np.savetxt('std.txt', std_values)

In [ ]:
plt.plot(np.arange(mean_values.shape[0]), mean_values[:, 0],label='Neural Network', color="red", lw=2) 

plt.fill_between(
    np.arange(mean_values.shape[0]),
    mean_values[:, 0] - std_values[:, 0],
    mean_values[:, 0] + std_values[:, 0],
    color="red",
    alpha=0.3 
)
plt.plot(np.arange(mean_values.shape[0]), mean_values[:, 1], label='Mutual Information', color="green", lw=2) 
plt.fill_between(
    np.arange(mean_values.shape[0]),
    mean_values[:, 1] - std_values[:, 1],
    mean_values[:, 1] + std_values[:, 1],
    color="green",
    alpha=0.3 
)
plt.plot(np.arange(mean_values.shape[0]), mean_values[:, 2], label='Random forest', color="blue", lw=2) 
plt.fill_between(
    np.arange(mean_values.shape[0]),
    mean_values[:, 2] - std_values[:, 2],
    mean_values[:, 2] + std_values[:, 2],
    color="blue",
    alpha=0.3 
)


plt.xlabel('Positions')
plt.ylabel('Scores')
plt.title('Position importances')
plt.legend()

plt.show()
